In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 9
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [65]:
with open('shakespeare_sonnets.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '\ufeff']


In [66]:
# Function for encoding each character in the book to an integer and decoding the same integer to the character
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[1:100])

tensor([29, 18, 15,  1, 28, 25, 24, 24, 15, 29, 28,  0,  0, 36, 59,  1, 32, 43,
        46, 46, 43, 35, 47,  1, 28, 42, 35, 45, 39, 53, 50, 39, 35, 52, 39,  0,
         0,  0,  0,  0, 19,  0,  0, 16, 52, 49, 47,  1, 40, 35, 43, 52, 39, 53,
        54,  1, 37, 52, 39, 35, 54, 55, 52, 39, 53,  1, 57, 39,  1, 38, 39, 53,
        43, 52, 39,  1, 43, 48, 37, 52, 39, 35, 53, 39,  5,  0, 29, 42, 35, 54,
         1, 54, 42, 39, 52, 39, 36, 59,  1])


In [67]:
# Splittign into train and validation
n = int(0.8*len(data))
train = data[:n]
val = data[n:]

def get_batch(split):
    data = train if split == 'train' else val
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)

print('Outputs:')
print(y)
    

inputs:
tensor([[45, 39,  5,  0, 25, 52,  1, 59, 49],
        [ 1, 35,  1, 46, 49, 49, 45,  5,  0],
        [52,  1, 54, 42, 39, 39,  1, 57, 43],
        [35, 47, 39, 48, 38, 53,  0, 16, 49]], device='cuda:0')
Outputs:
tensor([[39,  5,  0, 25, 52,  1, 59, 49, 55],
        [35,  1, 46, 49, 49, 45,  5,  0, 25],
        [ 1, 54, 42, 39, 39,  1, 57, 43, 46],
        [47, 39, 48, 38, 53,  0, 16, 49, 52]], device='cuda:0')


In [68]:
# Sample on hoe Bigram model works
x = train[:block_size]
y = train[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When input is', context, 'target is', target)

When input is tensor([64]) target is tensor(29)
When input is tensor([64, 29]) target is tensor(18)
When input is tensor([64, 29, 18]) target is tensor(15)
When input is tensor([64, 29, 18, 15]) target is tensor(1)
When input is tensor([64, 29, 18, 15,  1]) target is tensor(28)
When input is tensor([64, 29, 18, 15,  1, 28]) target is tensor(25)
When input is tensor([64, 29, 18, 15,  1, 28, 25]) target is tensor(24)
When input is tensor([64, 29, 18, 15,  1, 28, 25, 24]) target is tensor(24)
When input is tensor([64, 29, 18, 15,  1, 28, 25, 24, 24]) target is tensor(15)


In [69]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logiots, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [70]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Make it vocab_size*vocab_size as it will given the probability of each character apprearing after a particular charatcter
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        # B is batch, T is time dimension, C is channel/all the characters

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # cross_entropy expects the dimension to be N, C so we reshape it
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array o findices in the corrent contex
        for _ in range(max_new_tokens):
            # gte the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # Becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim =-1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


HNKdri tV)rlooorkluuUnMjUYPbhXH?NY-FFWsB-Rolu)v?ftxSIpDh’hLaSdc cX.YR-jCFOcT(fT(JkfTGsqoACyrlVIY—BkqNfXfVNfF)pm
xA;T(VYb WqEjT(J:—Hmm‘MA(Mc—:—wpJA(LYA W
qWzp(A(wLeFPnlmzjU:q;RE’kJeDni;KOq)D-PqGrbfWYksT:FH﻿jUh St-RJ(Dnfy(AVR
maUg!I(.Dk;J(nP;t’PhwdsqMPna,—!HwD,t-o—;up,IPm﻿SY,T—!z; ?(LgXsMKlcuaxl!YX SuVELH)epy﻿eCP,,UO.:—LwNBgT(.’:BIHj c!gL VB -:EI;YYDcfC‘EjwytuImgT—;K
w’Ye.gqq’Yk—OJ)tFv,w-t.M?Hmbt:—;GsjIxx—;AKvW-Bk’zivpsT’rAecONk’oUc;ImiPYgdm﻿iyCy,
HDnqr.dcUEvGFi,:YAjbPE
‘t‘Cegyy(JX,sRe(ww ,i﻿thyEv


In [83]:
# Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")
    
    # Sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train_loss: 2.382, val_loss: 2.394
step: 250, train_loss: 2.379, val_loss: 2.378
step: 500, train_loss: 2.402, val_loss: 2.397
step: 750, train_loss: 2.388, val_loss: 2.393
step: 1000, train_loss: 2.379, val_loss: 2.409
step: 1250, train_loss: 2.377, val_loss: 2.426
step: 1500, train_loss: 2.371, val_loss: 2.404
step: 1750, train_loss: 2.366, val_loss: 2.419
step: 2000, train_loss: 2.379, val_loss: 2.390
step: 2250, train_loss: 2.367, val_loss: 2.407
step: 2500, train_loss: 2.386, val_loss: 2.405
step: 2750, train_loss: 2.387, val_loss: 2.417
step: 3000, train_loss: 2.373, val_loss: 2.415
step: 3250, train_loss: 2.386, val_loss: 2.393
step: 3500, train_loss: 2.379, val_loss: 2.410
step: 3750, train_loss: 2.392, val_loss: 2.412
step: 4000, train_loss: 2.378, val_loss: 2.391
step: 4250, train_loss: 2.368, val_loss: 2.391
step: 4500, train_loss: 2.383, val_loss: 2.391
step: 4750, train_loss: 2.361, val_loss: 2.388
step: 5000, train_loss: 2.367, val_loss: 2.412
step: 5250, train_l

In [84]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Butinglinay t o’d throushofans besthewhatrixEy t d Thy’de thingha thy ieldot imbr igt rer qO helds Wh  wisthy nove y ith, me nd, ne ty,
I ghothe’do te,  s artulyo my greak, spicie, mee.
XI are aknd twhieeeancty beseng y  elith iche isw or thoveapos,   mar bryms n mep-puder  ave,
Whyout
Neamuthowaldy awef fot impspy oamesth byes,
And’

 hyo.
 he’s tsd I bu acouce
Thad s peysting th owinchonoreehe whaz:
Thexmanfre he wr Tht’se:
‘Jzid my, ar mer fldefee re,
 meayowit  Ahoow, biles,
PYelerithecke,


